In [1]:
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage import io, transform, color
from skimage.feature import hog
from skimage import exposure
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import os
from tqdm import tqdm
import cv2

import shutil

### downlaod the data from kaggke

In [2]:


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [3]:
!kaggle competitions download -c umor-classification

100% 192M/193M [00:11<00:00, 16.4MB/s]
100% 193M/193M [00:11<00:00, 17.6MB/s]


In [4]:
!unzip umor-classification.zip

Archive:  umor-classification.zip
  inflating: images/images/1006.png  
  inflating: images/images/1028.png  
  inflating: images/images/1046.png  
  inflating: images/images/1062.png  
  inflating: images/images/1066.png  
  inflating: images/images/1068.png  
  inflating: images/images/1070.png  
  inflating: images/images/1073.png  
  inflating: images/images/1091.png  
  inflating: images/images/1097.png  
  inflating: images/images/1105.png  
  inflating: images/images/1127.png  
  inflating: images/images/1138.png  
  inflating: images/images/1154.png  
  inflating: images/images/1156.png  
  inflating: images/images/1159.png  
  inflating: images/images/1183.png  
  inflating: images/images/1191.png  
  inflating: images/images/1202.png  
  inflating: images/images/1218.png  
  inflating: images/images/1230.png  
  inflating: images/images/1234.png  
  inflating: images/images/1235.png  
  inflating: images/images/1240.png  
  inflating: images/images/1253.png  
  inflating: ima

***

### read the image path

In [5]:
# # Load and preprocess training images
image_folder   = '/content/images/images/'

In [6]:
print(f"There are {len(os.listdir(image_folder ))} images in  dataset")

There are 775 images in  dataset


***

### read the csv files

In [7]:
train_df  = pd.read_csv('trainset.csv')
test_df   = pd.read_csv('testset.csv')

In [8]:
train_df['Class'].value_counts()

Benign       300
Malignant    148
Normal        91
Name: Class, dtype: int64

In [9]:
train_df

,Unnamed: 0,Id,Class
0,211,8602,Malignant
1,315,7464,Malignant
2,165,5832,Malignant
3,188,2843,Malignant
4,710,5272,Benign
...,...,...,...
534,763,8176,Benign
535,192,5434,Malignant
536,629,5366,Benign
537,559,9811,Benign


***
#### read the image folder

In [10]:
images = os.listdir(image_folder)

In [11]:
id_to_label_train = dict(zip(train_df['Id'], train_df['Class']))

***
##### read the train folder

In [12]:
# create a list for both images and labels
images_train = []
labels_train = []

In [13]:
for index, row in train_df.iterrows():
  im_id = row['Id']
  if im_id in id_to_label_train :
    label = id_to_label_train [im_id]
    image_path = os.path.join(image_folder, f"{im_id}.png")
    if os.path.exists(image_path):  # Check if the image file exists
        image = cv2.imread(image_path)  # Load the image using cv2

        if image is not None:  # Check if the image was loaded successfully
            images_train.append(image)
            labels_train.append(label)
            print(labels_train)

['Malignant']
['Malignant', 'Malignant']
['Malignant', 'Malignant', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant', 'Benign', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant', 'Benign', 'Benign', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignan

In [14]:
X_train = np.array(images_train)
X_train.shape

<ipython-input-14-8020449d68be>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(images_train)


(539,)

In [15]:
y_train = np.array(labels_train)
y_train.shape

(539,)

***
##### read the test folder

In [16]:
# Get the image IDs from the test CSV
test_image_ids = list(test_df['Id'])

In [17]:
image_test = []
for index, row in test_df.iterrows():
  image_id = row['Id']
  if image_id in test_image_ids:
    image_path = os.path.join(image_folder, f"{image_id}.png")
    # Check if the image file exists
    if os.path.exists(image_path):
      # Load the image using cv2
      image = cv2.imread(image_path)
      image_test.append(image)





In [18]:
image_test

[array([[[151, 151, 151],
         [ 11,  11,  11],
         [  0,   0,   0],
         ...,
         [169, 169, 169],
         [170, 170, 170],
         [169, 169, 169]],
 
        [[  0,   0,   0],
         [  7,   7,   7],
         [  1,   1,   1],
         ...,
         [152, 152, 152],
         [149, 149, 149],
         [146, 146, 146]],
 
        [[  1,   1,   1],
         [  0,   0,   0],
         [138, 138, 138],
         ...,
         [153, 153, 153],
         [153, 153, 153],
         [152, 152, 152]],
 
        ...,
 
        [[ 48,  48,  48],
         [ 49,  49,  49],
         [ 50,  50,  50],
         ...,
         [ 42,  42,  42],
         [ 42,  42,  42],
         [ 42,  42,  42]],
 
        [[ 45,  45,  45],
         [ 46,  46,  46],
         [ 46,  46,  46],
         ...,
         [ 42,  42,  42],
         [ 44,  44,  44],
         [ 44,  44,  44]],
 
        [[ 43,  43,  43],
         [ 43,  43,  43],
         [ 43,  43,  43],
         ...,
         [ 43,  43,  43],
  

In [19]:
X_test = np.array(image_test)
X_test.shape

<ipython-input-19-71dd8b3a7a86>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array(image_test)


(229,)

***
know more information about the data


In [20]:
# summarize dataset shape
print('Train', X_train.shape, y_train.shape)
print('Test', (X_test.shape))
# summarize dataset shape
print('Train for image', X_train[0].shape, y_train[0].shape)
print('Test for image', (X_test[0].shape))

Train (539,) (539,)
Test (229,)
Train for image (470, 564, 3) ()
Test for image (467, 551, 3)


In [21]:
type(X_train)

numpy.ndarray

***

to here we will start prepraring data
***


### resize the image

for train

In [22]:
target_size = (200, 200)  # Replace with your desired target size

# Initialize an empty list to store resized images
resized_images = []

# Loop through each image in X_train and resize
for image in X_train:
    resized_image = cv2.resize(image, target_size)
    resized_images.append(resized_image)

# Convert the list of resized images back to a NumPy array
X_train = np.array(resized_images)
# Now X_train_resized contains the resized images
print("Shape of X_train_resized:", X_train.shape)

Shape of X_train_resized: (539, 200, 200, 3)


for test

In [23]:
target_size = (200, 200)  # Replace with your desired target size

# Initialize an empty list to store resized images
resized_images = []

# Loop through each image in X_test and resize
for image in X_test:
    resized_image = cv2.resize(image, target_size)
    resized_images.append(resized_image)

# Convert the list of resized images back to a NumPy array
X_test = np.array(resized_images)
# Now X_test contains the resized images
print("Shape of X_test:", X_test.shape)

Shape of X_test: (229, 200, 200, 3)


***
shape min max std mean for X_train and X_test


In [24]:
# Now compute the minimum and maximum values
minimum_pixel_value_train = np.min(X_train)
maximum_pixel_value_train = np.max(X_train)
print("Train - Minimum Pixel Value:", minimum_pixel_value_train)
print("Train - Maximum Pixel Value:", maximum_pixel_value_train)

minimum_pixel_value_test = np.min(X_test)
maximum_pixel_value_test = np.max(X_test)
print("Test - Minimum Pixel Value:", minimum_pixel_value_test)
print("Test - Maximum Pixel Value:", maximum_pixel_value_test)

Train - Minimum Pixel Value: 0
Train - Maximum Pixel Value: 255
Test - Minimum Pixel Value: 0
Test - Maximum Pixel Value: 255


### split the train into train and val set

In [25]:
# Split data into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [26]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_train, return_counts=True)
print("Class Frequencies in y_train:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

Class Frequencies in y_train:
Class Benign: 243 samples
Class Malignant: 118 samples
Class Normal: 70 samples


In [27]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_val, return_counts=True)
print("Class Frequencies in y_train:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

Class Frequencies in y_train:
Class Benign: 57 samples
Class Malignant: 30 samples
Class Normal: 21 samples


In [28]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((431, 200, 200, 3), (108, 200, 200, 3), (431,), (108,))

In [47]:
import numpy as np
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Reshape and normalize the image data
X_train_scaled = X_train / 255.0
X_val_scaled = X_val / 255.0

# Convert images to grayscale
X_train_gray = np.array([rgb2gray(image) for image in X_train_scaled])
X_val_gray = np.array([rgb2gray(image) for image in X_val_scaled])

# Feature extraction using HOG for training data
hog_features_train = []
for image in X_train_gray:
    hog_features = hog(image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    hog_features_train.append(hog_features)
hog_features_train = np.array(hog_features_train)

# Feature extraction using HOG for validation data
hog_features_val = []
for image in X_val_gray:
    hog_features = hog(image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    hog_features_val.append(hog_features)
hog_features_val = np.array(hog_features_val)

# Scale features using Min-Max scaling
scaler = MinMaxScaler()
scaled_features_train = scaler.fit_transform(hog_features_train)
scaled_features_val = scaler.transform(hog_features_val)



In [48]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

svm_classifier = SVC(kernel='rbf', C=100,gamma= 0.001)
svm_classifier.fit(scaled_features_train, y_train)

# Make predictions
y_pred = svm_classifier.predict(scaled_features_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Generate classification report
class_report = classification_report(y_val, y_pred)
print("Classification Report:\n", class_report)

Accuracy: 0.6851851851851852
Confusion Matrix:
 [[49  5  3]
 [14 14  2]
 [ 9  1 11]]
Classification Report:
               precision    recall  f1-score   support

      Benign       0.68      0.86      0.76        57
   Malignant       0.70      0.47      0.56        30
      Normal       0.69      0.52      0.59        21

    accuracy                           0.69       108
   macro avg       0.69      0.62      0.64       108
weighted avg       0.69      0.69      0.67       108

